In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import os
import joblib
import requests
from predict import predict_price, predict_tranche  # ML local
from tensorflow.keras.models import load_model
from sentence_transformers import SentenceTransformer


c:\wamp64\www\ia.follaco.fr\.venv\lib\site-packages\gradio_client\documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
c:\wamp64\www\ia.follaco.fr\.venv\lib\site-packages\gradio_client\documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


In [2]:

# 📂 Chemins
LOG_PATH = "flagged/log.csv"
API_URL = "http://127.0.0.1:8000/predict"
choices = {
    "Acceptable": 80,
    "Moyenne": 85,
    "Bonne": 90,
    "Très Bonne": 96,
    "Excellente": 99
}
niveau_mapping = ["Beginner", "Intermediate", "Expert"]  # pour DL local

# 🧠 Chargement lazy des modèles DL
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
deep_model = None
scaler_dl = None

def load_deep_models():
    global deep_model, scaler_dl
    if deep_model is None:
        deep_model = load_model("models/deep/deep_model.h5")
        scaler_dl = joblib.load("models/deep/scaler.pkl")

def faire_une_prediction(description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, modele):
    fiabilite = (choices[fiabilite_choix] if use_predefined else fiabilite_percent) / 100

    try:
        if modele == "ML - Local":
            prix = predict_price(description, fiabilite)
            tranche = predict_tranche(description, fiabilite)

        elif modele == "DL - Local":
            load_deep_models()
            emb = embedding_model.encode([description]).flatten()
            niveau_ohe = [1 if niveau == n else 0 for n in niveau_mapping]
            features = np.hstack([emb, niveau_ohe, [fiabilite]])
            features_scaled = scaler_dl.transform([features])
            prix = deep_model.predict(features_scaled)[0][0]
            tranche = "Non évaluée"

        elif modele == "API - FastAPI":
            response = requests.post(API_URL, json={
                "Description": description,
                "Niveau": niveau,
                "Fiabilite": fiabilite
            }, timeout=5)
            response.raise_for_status()
            prix = response.json().get("prix_predit", -1)
            tranche = "Non évaluée"

        else:
            return "❌ Modèle inconnu", ""

        return round(prix, 2), tranche

    except Exception as e:
        return f"Erreur : {str(e)}", ""

def enregistrer_log(description, use_predefined, fiabilite_percent, fiabilite_choix, prix, tranche, modele):
    fiabilite = choices[fiabilite_choix] if use_predefined else fiabilite_percent
    log_data = {
        "Description": description,
        "Fiabilité (%)": fiabilite,
        "Prix prédit (€)": prix,
        "Tranche prédite": tranche,
        "Modèle utilisé": modele
    }

    df_log = pd.DataFrame([log_data])
    if os.path.exists(LOG_PATH):
        df_log.to_csv(LOG_PATH, mode="a", index=False, header=False)
    else:
        df_log.to_csv(LOG_PATH, index=False)

    return "✅ Signalement enregistré avec succès."

with gr.Blocks() as iface:
    gr.Markdown("""
        ## 🎯 Application de prédiction de prix Fiverr
        Cette application permet d’estimer automatiquement :
        - 💰 Le **prix probable** d’un service publié sur Fiverr,
        - 📊 Sa **tranche de prix** parmi trois catégories (Basse / Moyenne / Haute).
        Elle s’appuie sur un pipeline hybride combinant des embeddings de description et des variables numériques.
        🔁 Vous pouvez maintenant choisir entre trois modèles : ML, Deep Learning, ou appel API REST FastAPI.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            description = gr.Textbox(label="✏️ Titre du service", value="Je fais le ménage")
            niveau = gr.Dropdown(label="🔰 Niveau du vendeur", choices=niveau_mapping, value="Beginner", visible=False)
            use_predefined = gr.Checkbox(label="🎛️ Utiliser les niveaux prédéfinis de fiabilité", value=True)

            fiabilite_percent = gr.Slider(label="Fiabilité (%)", minimum=0, maximum=100, value=80, step=5, visible=False)
            fiabilite_choix = gr.Radio(label="🎚️ Choisissez un niveau de fiabilité", choices=list(choices.keys()), value="Acceptable", visible=True)

            modele = gr.Radio(label="🧠 Choix du modèle", choices=["ML - Local", "DL - Local", "API - FastAPI"], value="ML - Local")

            def sync_slider_with_radio(choix):
                return gr.update(value=choices[choix])

            fiabilite_choix.change(sync_slider_with_radio, inputs=fiabilite_choix, outputs=fiabilite_percent)

            def toggle_inputs(use_predef):
                return {
                    fiabilite_percent: gr.update(visible=not use_predef),
                    fiabilite_choix: gr.update(visible=use_predef)
                }

            use_predefined.change(toggle_inputs, inputs=use_predefined, outputs=[fiabilite_percent, fiabilite_choix])
            bouton_predire = gr.Button("📈 Estimer le prix")

        with gr.Column(scale=1):
            sortie_prix = gr.Textbox(label="💰 Prix estimé")
            sortie_tranche = gr.Textbox(label="📊 Tranche estimée")
            bouton_signaler = gr.Button("🚨 Ajouter au fichier log.csv")
            confirmation = gr.Textbox(label="✅ Confirmation", visible=False)

            bouton_predire.click(
                fn=faire_une_prediction,
                inputs=[description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, modele],
                outputs=[sortie_prix, sortie_tranche]
            )

            bouton_signaler.click(
                fn=enregistrer_log,
                inputs=[description, use_predefined, fiabilite_percent, fiabilite_choix, sortie_prix, sortie_tranche, modele],
                outputs=confirmation
            )


# 🚀 Lancement
iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade.
--------


## 🧠 Justification des incohérences apparentes dans les tranches prédictives

### ❓ Problème observé

Dans certains cas, il peut sembler incohérent qu’une **tranche de prix estimée** soit plus élevée pour une **fiabilité "Acceptable"** que pour une fiabilité "Excellente". Par exemple :

- Pour une **fiabilité Acceptable (80%)**, la tranche prédite est **Haute**
- Pour une **fiabilité Excellente (99%)**, la tranche prédite est **Basse**

### 🧩 Explication technique

Cette situation peut être expliquée de façon cohérente par la conception du pipeline :

1. **Deux modèles indépendants sont utilisés** :
   - Un modèle de **régression** qui prédit un **prix numérique**
   - Un modèle de **classification** qui prédit une **tranche catégorielle** (`Basse`, `Moyenne`, `Haute`)
   - Ces deux modèles sont **entraînés séparément**, sur des cibles différentes

2. **La classification ne dépend pas du prix prédit** :
   - Le modèle de classification apprend à prédire une **catégorie** (la tranche réelle observée) en fonction d’un ensemble de variables, notamment :
     - L’embedding de la description
     - La fiabilité pondérée
   - Il ne connaît **ni le prix réel, ni le prix prédit par le modèle de régression**

3. **La fiabilité pondérée n’a pas une influence linéaire** :
   - Une meilleure fiabilité (Excellente) ne garantit pas forcément un prix plus élevé ni une tranche plus haute
   - En pratique, **le modèle peut estimer qu’un service générique très fiable** (par exemple un intitulé trop large ou basique) appartient aux tranches basses
   - Tandis qu’un **service plus ciblé ou spécialisé** avec une fiabilité moyenne peut être associé aux tranches hautes

### 📌 Interprétation métier

Ce comportement n'est pas une erreur mais une **manifestation de la logique d'apprentissage supervisé** :

> Le modèle de classification apprend des **règles implicites** présentes dans les données d’entraînement. Il peut considérer certaines descriptions comme typiquement premium, indépendamment du niveau de fiabilité, ce qui entraîne une tranche "Haute" même avec une fiabilité modeste.

### ✅ Conclusion

Il est important de retenir :

- Le système n’utilise **pas une simple règle à seuils** sur le prix pour classer les tranches
- Il s’agit d’un modèle **entraîné à partir d’observations réelles**
- Les incohérences apparentes sont souvent dues à des **corrélations non triviales** dans les données (nature du service, vocabulaire utilisé, etc.)

Cette explication souligne la complexité du comportement des modèles d’IA, et la nécessité de les évaluer **sur l’ensemble du pipeline**, pas uniquement sur un cas particulier.

In [3]:
# Exemple de description et niveau à tester
desc = "Je fais le ménage"

# Test de l’impact de la fiabilité sur la prédiction du prix
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]:
    prix = predict_price(desc, f)
    print(f"Fiabilité = {f*100:.0f}% → Prix prédit = {prix} €")

Fiabilité = 0% → Prix prédit = 3.45 €
Fiabilité = 10% → Prix prédit = 3.45 €
Fiabilité = 20% → Prix prédit = 3.45 €
Fiabilité = 30% → Prix prédit = 3.45 €
Fiabilité = 40% → Prix prédit = 3.45 €
Fiabilité = 50% → Prix prédit = 3.45 €
Fiabilité = 60% → Prix prédit = 3.45 €
Fiabilité = 70% → Prix prédit = 3.45 €
Fiabilité = 80% → Prix prédit = 3.45 €
Fiabilité = 85% → Prix prédit = 3.55 €
Fiabilité = 90% → Prix prédit = 3.72 €
Fiabilité = 91% → Prix prédit = 3.72 €
Fiabilité = 92% → Prix prédit = 3.72 €
Fiabilité = 93% → Prix prédit = 3.72 €
Fiabilité = 94% → Prix prédit = 3.84 €
Fiabilité = 95% → Prix prédit = 3.97 €
Fiabilité = 96% → Prix prédit = 4.04 €
Fiabilité = 97% → Prix prédit = 4.19 €
Fiabilité = 98% → Prix prédit = 4.33 €
Fiabilité = 99% → Prix prédit = 4.33 €
Fiabilité = 100% → Prix prédit = 4.3 €
